In [1]:
import gc
import os

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression as LR
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
pd.set_option('display.max_rows', 500)

from valkyrie.avalon.feature_analyzer import FeatureMgr
from valkyrie.securities import stocks_good_dvd, parent
from valkyrie.quants import lm_fit

/tmp/ipykernel_263317/1962559675.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load Data

In [ ]:
#feature_mgr = FeatureMgr('20210101', '20210630', ['TWO PRC'] , 'latest')#
feature_mgr = FeatureMgr('20210101', '20210630', stocks_good_dvd() , 'eve_features3_2x0_3d')
ret_ns = [int(1 * 2 * 60 * 6.5), int(3 * 2 * 60 * 6.5), int(5 * 2 * 60 * 6.5), int(10 * 2 * 60 * 6.5)]

ret_col, s2_col = 'risk_tv', 'cywa_mkt_s2h'
df_tv = feature_mgr.calc_rets('log', ret_col, ret_ns, s2_col)
df_tv['parent'] = df_tv['ticker'].apply(parent)

ret_col, s2_col = 'cywa_mkt_xh', 'cywa_mkt_s2h'
df_cy = feature_mgr.calc_rets('log', ret_col, ret_ns, s2_col)
df_cy['parent'] = df_cy['ticker'].apply(parent)
gc.collect()

In [ ]:
sim_dir = '/home/bb/projects/valkyrie/sim_dir/eve_features3_2x0_3dB/summary/fills/20210216'

In [ ]:
df = []
for fn in os.listdir(sim_dir):    
    name = fn.split('.')[0]
    df_temp = pd.read_hdf(f'{sim_dir}/{fn}')                
    df.append(df_temp)
df = pd.concat(df)

In [ ]:
[c for c in list(df_tv.columns) if 'feature_' in c]

In [ ]:
df_tv['cy_pred'] = df_tv 'feature_cluster','feature_long_dpff','feature_mkt_dpff'

In [ ]:
0.0001 /  0.07 * 25

In [ ]:
df_tv.query('ticker == "NRZ PRA" and date == "20210216"')[['feature_cluster']].plot(alpha = 0.6)

In [ ]:
df[['edge']].sum()

In [ ]:
df.sort_values('edge', ascending=False)

In [ ]:
df = pd.DataFrame(name2df, index = 'name')

In [ ]:
df

# Checking

### against nbbo

In [ ]:
hls = [1, 60, 600, 3600]
print(f'Within bbo %')
tv_cols = [c for c in df_tv if 'cywa_' in c and '_tv' in c and 'pff_' not in c] + ['tv']
df_against_nbbo = []
for col in ['risk_tv'] + tv_cols:
    below = df_tv.eval(f'{col} <= bid ').sum() / df_tv.shape[0]
    above = df_tv.eval(f'{col} >= ask ').sum() / df_tv.shape[0]
    df_against_nbbo.append({'name': col, 'below' : below, 'above' : above})
df_against_nbbo = pd.DataFrame(df_against_nbbo).set_index('name')
print(df_against_nbbo)

### tv stats

In [ ]:
for col in tv_cols:    
    df_tv[f'drt_{col}'] = df_tv.eval(f'{col} - risk_tv')
dtv_cols = [c for c in df_tv if 'drt_' in c] 
cols = tv_cols + dtv_cols
pd.concat( [pd.DataFrame(df_tv[cols].diff().std(), columns=['diff_std']), df_tv[cols].describe(percentiles = [0.01, 0.5,0.99]).T],axis= 1)

### dtv corr

In [ ]:
ret_cols = [f'risk_tv_ret_{ret}_n' for ret in ret_ns]
dtv_cols = [c for c in df_tv if 'drt_' in c] 
cols = dtv_cols + ret_cols
df_tv[cols].corr()[ret_cols + dtv_cols].query('index.str.contains("drt_")')#[ret_cols + dtv_cols]

In [ ]:
#weighted correlation
from statsmodels.stats.weightstats import DescrStatsW
wcoeff = {}
for n in ret_ns:
    X = df_tv[['drt_tv', f'risk_tv_ret_{n}_n']].values
    dsw = DescrStatsW(X, weights=df_tv[f'wgt_risk_tv_ret_{n}_n'].values)
    wcoeff[f'drt_tv_vs_risk_tv_ret_{n}_n'] = dsw.corrcoef[0,1]
pd.DataFrame(wcoeff, index = ['corr'])

# Check by name

In [ ]:
df.query('ticker == "AGNCM" and date == "20210104"')[['bid','ask','risk_tv']].plot()

In [ ]:
df.query('ticker == "AGNCM"')['bid ask tv'.split() + [c for c in df if 'cywa_10' in c]].head(5000).to_csv('/home/bb/tmp/tmp.csv')

In [ ]:
df.query('ticker == "TWO PRC"')['bid ask tv'.split()].head(5000).plot(alpha = 0.7)

In [ ]:
df.query('ticker == "BAC PRB" and "20210428" <= date <= "20210429"')[['cywa_mkt_s2h']].plot(alpha = 0.7)
df.query('ticker == "BAC PRB" and "20210428" <= date <= "20210429"')[['bid','ask','risk_tv']].plot(alpha = 0.7)

In [ ]:
xcols = [c for c in df if 'x_' in c]
df[xcols + ['mid_ret_780_n']].corr()

In [ ]:
df[['risk_tv_ret_780_n', 'cywa_10_ret_780_n', 'mid_ret_780_n']].describe()

In [ ]:
xcols = ['intercept', 'auto_short_minus_cx', 'auto_long_minus_short']
#xcols = []
xcols += ['PFF_cur_diff', 'PFF_short_hl_diff', 'PFF_long_hl_diff']

In [ ]:
df[ycols].describe()

# Combine Features

In [ ]:
ret_ns

In [ ]:
cywa_hl = 'long'
yhz = 2340


df['feature_cluster'] = df.eval(f'cluster_{cywa_hl}_avg_xh - (cywa_{cywa_hl}_xh - cywa_mkt_xh)')                                  
df['feature_long_dpff'] = df.eval(f'pff_cywa_{cywa_hl}_xh - cywa_{cywa_hl}_xh')
df['feature_mkt_dpff'] = df.eval(f'pff_cywa_mkt_xh - cywa_mkt_xh ')
df['total_s2'] = df.eval(f'cywa_mkt_s2h + cywa_{cywa_hl}_s2h + cluster_{cywa_hl}_avg_s2 + (1.0 / wgt_cywa_mkt_xh_ret_{yhz}_n)')
df['wgt'] = df.eval('1.0 / total_s2')

df_fit = df.query('abs(feature_cluster) >= 0.0 * 1.0e-4 or abs(feature_long_dpff - feature_mkt_dpff) >= 0.0 * 1.0e-4').copy()
print(f'{df_fit.shape[0]/df.shape[0]}')

xcols = [c for c in df if 'feature_' in c]
ycols = [f'cywa_mkt_xh_ret_{yhz}_n']
#xcols = ['feature_cluster']
xcols = [c for c in df_fit if 'feature_' in c]
res = lm_fit(df_fit, xcols, ycols, ['wgt'])
res

In [ ]:
zz = res.loc['model']

In [ ]:
zz.lr

In [ ]:
res.to_csv('/home/bb/projects/valkyrie/configs/coeffs_tmp.txt')

In [ ]:
2+3

In [ ]:
df_cy['date'].unique()

In [ ]:
df_test = df_cy.query('date == "20210301"').copy()

In [ ]:
df_test['edge_abs'] = df_test.eval( '(tv > ask) * (tv - ask) + (bid > tv) * (bid - tv)')

In [ ]:
df_test.query('edge_abs > 0').sort_values('edge_abs', ascending=False)['ticker tv bid 	bz 	ask 	az edge_abs'.split()]

In [ ]:
df_test.query('ticker == "AGNCM"')['bid tv ask'.split()].plot()